# Assignment 3 | Sentiment Analyisis

### Relevant Libraries

In [1]:
#libraries
from collections import Counter
import pandas as pd
import numpy as np
from prettytable import PrettyTable
import seaborn as sns
import matplotlib.pyplot as plt
import re
import regex
import string
from sklearn.feature_extraction.text import CountVectorizer
from emosent import get_emoji_sentiment_rank

from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
#installing tweet-preprocessor
from tqdm.auto import tqdm
!pip install tweet-preprocessor --user
import preprocessor as p
from pathlib import Path
!pip install emosent-py --user
#emoji library
!pip install emoji --user
import emoji

# model save
import joblib

#split
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /home/hemant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/hemant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hemant/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/hemant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/hemant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hemant/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
import re

In [3]:
class featurise(object):
    def __init__(self, x,y):
        super(featurise, self).__init__()
        self.seq = None
        self.x = x
        self.y = y
        ##########################################
        self.final_df = pd.DataFrame()
        self.final_df['tweets'] = x
        self.final_df['target'] = y
        ##########################################
        self.negation_w = set(['never', 'no', 'nothing', 'nowhere', 'noone', 'none', 'not', 'havent', 'hasnt', 'hadnt', 'cant', 
                         'couldnt', 'shouldnt', 'wont', 'wouldnt', 'dont', 'doesnt', 'didnt', 'isnt', 'arent', 'aint'] )
        self.stop_words = stopwords.words('english')
        self.stop_words.remove('not')
        
        self.nrc = pd.read_csv("/home/hemant/NLP_LECTURE/Assignment_3/data/3a/lexicons/7. NRC-Hashtag-Sentiment-Lexicon-v0.1/unigrams-pmilexicon.txt", header=None, sep="\t")
        self.dic = {self.nrc[0][i]:self.nrc[1][i] for i in range(len(self.nrc))}
        self.mpqa_df = pd.read_csv("/home/hemant/NLP_LECTURE/Assignment_3/data/3a/lexicons/mpqa.txt",header=None, sep="\t")
        self.dic_mpqa = self.mpqa_df.set_index(0).to_dict()
        self.bingliu_df = pd.read_csv("/home/hemant/NLP_LECTURE/Assignment_3/data/3a/lexicons/1. BingLiu.csv", header=None, sep="\t")
        self.dic_bing = self.bingliu_df.set_index(0).to_dict()
        self.sent140_df = pd.read_csv("/home/hemant/NLP_LECTURE/Assignment_3/data/3a/lexicons/3. Sentiment140-Lexicon-v0.1/unigrams-pmilexicon.txt", header=None, sep="\t")
        self.dic_sent = self.sent140_df.set_index(0).to_dict()

    def elongated_word(self, x):
        count = 0
        regex_ = re.compile(r"(.)\1{2}")
        for i in x.split():            
            if regex_.search(i): count+=1
        return count
        
    def emoji(self, x):
        emo_final = []
        for word_ in tqdm(x):
            emoji_ = {d:word for d,word in enumerate(regex.findall(r'\X', word_)) if any(char in emoji.UNICODE_EMOJI for char in word)}

            dummy = [0,0,0]
            for i,e in emoji_.items():
                if i == len(word_) - 1:
                    dummy[-1] += 1
                    continue
                elif get_emoji_sentiment_rank(e)['positive'] > get_emoji_sentiment_rank(e)['negative']: dummy[0] += 1
                elif get_emoji_sentiment_rank(e)['positive'] < get_emoji_sentiment_rank(e)['negative']: dummy[1] += 1
            emo_final.append(dummy)
        emo_final = np.array(emo_final)
        self.final_df['p+'] = emo_final[:,0]
        self.final_df['p-'] = emo_final[:,1]
        self.final_df['p+-'] = emo_final[:,2]
    
    def punctuation(self, x):
        regex = re.compile(r"([?|!])\1{0}")
        dummy = []
        for i in tqdm(x):
            d = [len([word for word in i.split() if regex.search(word)])]
            if regex.search(i.split()[-1]): d.append(1)
            else: d.append(0)
            dummy.append(d)
        dummy = np.array(dummy)
        self.final_df['punctu_1'] = dummy[:,0]
        self.final_df['punctu_2'] = dummy[:,1]
                   
    def all_caps(self, x):
        return len([1 for each in x.split() if each.isupper()])

    def hashtag_count(self, x):
        return len([1 for word in x.split() if word.startswith('#') and len(word)>1 ])
    
    def negation(self, x):
        flag = 0
        splits = ",.:;!?"
        final = []
        count = 0
#         x = f"I don't think I will enjoy it: it might be too spicy.".lower()
        for word in x.split():
            if flag ==1:
                for symb in splits:
                    if symb in word:
                        flag = 0
                        word_split = word.split(symb)
                        
                        if len(word_split[0])>0:
                        
                            count+=1
                        
                            if len(word_split[-1])>1:
                        
                                pass
                        else:
                            pass
                if flag==1:
                    
                    count+=1
                
            elif flag==0:
                
                pass

            clean = re.sub('[^A-Za-z0-9]+', '', word)
            if clean in self.negation_w:
                flag=1
        return count

    def pos_tag_feat(self, a):
        print('pos_tags')
        tagged_texts = [pos_tag_sents([i.split()])[0] for i in tqdm(a)]

        dummy = [Counter([j[-1] for j in i]) for i in tqdm(tagged_texts)]
        pos_vocab = set([j for i in dummy for j in i.keys()])
        pos_t_i = {i:d for d,i in enumerate(pos_vocab)}
        a = np.zeros((len(tagged_texts),len(pos_vocab)))

        for d,i in tqdm(enumerate(dummy)):
            for k,v in i.items():
                a[d][pos_t_i[k]] = v
        for d,i in enumerate(pos_t_i.keys()):
            self.final_df[i] = a[:,d]
#         return pd.DataFrame(a,columns=pos_t_i.keys())

    #########################
    
    def remove_stopword(self, x):
        dummy = []
        for i in x.split():
            if i not in self.stop_words: dummy.append(i)
        return ' '.join(dummy)
    
    def all_lower(self, x):
        return x.lower()
    def remove_punctuation(self, x):
        
        return re.sub(r'[^\w\s]', '', x)
    def remove_number(self, x):
        return ''.join([_ for _ in x if not _.isdigit()])
    def rem_un_url(self, x):
        # return p.clean(x)
        result = re.sub(r"http\S+", "", x)
        return re.sub('@[^\s]+','',result)

    #################################################################3
    def nrc_emotion_score(self,x):

        score = 0
        for each in  x.lower().split():
            if each in self.dic: score += self.dic[each]
        return score
    
    def polar_word_count_mpqa(self, x):

        pos,neg = 0,0
        for each in  x.split():
            if each.lower() in self.dic_mpqa[1]:
                if self.dic_mpqa[1][each.lower()] == 'negative': neg+=1
                else: pos+=1
        return pos,neg
    
    def polar_word_count_bingliu(self, x):
        pos,neg = 0,0
        for each in  x.split():
            if each.lower() in self.dic_bing[1]:
                if self.dic_bing[1][each.lower()] == 'negative': neg+=1
                else: pos+=1
        return pos,neg

    
    def agg_polarity_score_sent140(self,x):
        
        pos,neg = 0,0
        for each in  x.lower().split():
            if each in self.dic_sent[2]:
                pos+=self.dic_sent[2][each]
            if each in self.dic_sent[3]:
                neg+=self.dic_sent[3][each]
        return pos, neg
    
    def process(self):
        self.final_df['elongated'] = tqdm(self.x.map(self.elongated_word))
        self.emoji(list(self.x))
        self.punctuation(list(self.x))

        self.final_df['allcaps'] = [self.all_caps(i)  for i in tqdm(self.x)]
        self.final_df['hashtag'] = [self.hashtag_count(i)  for i in tqdm(self.x)]
        self.final_df['negation'] = [self.negation(i)  for i in tqdm(self.x)]
        self.final_df['negation'] = [self.negation(i)  for i in tqdm(self.x)]
        
        print('shit')
        self.final_df['nrc_emotion_score'] = [self.nrc_emotion_score(i) for i in tqdm(self.x)]
        dummy = np.array([self.polar_word_count_mpqa(i) for i in tqdm(self.x)])
        self.final_df["pos_mpqa"] = dummy[:,0]
        self.final_df["neg_mpqa"] = dummy[:,1]
        dummy = np.array([self.polar_word_count_bingliu(i) for i in tqdm(self.x)])
        self.final_df['pos_bilingu'] = dummy[:,0]
        self.final_df['neg_bilingu'] = dummy[:,1]
        dummy = np.array([self.agg_polarity_score_sent140(i) for i in tqdm(self.x)])
        self.final_df['pos_sent140'] = dummy[:,0]
        self.final_df['neg_sent140'] = dummy[:,1]

        
        self.final_df['new_tweets'] = [self.rem_un_url(i)  for i in tqdm(self.x)]
        self.final_df['new_tweets'] = [self.all_lower(i) for i in tqdm(self.final_df['new_tweets'])]
        self.final_df['new_tweets'] = [self.remove_number(i) for i in tqdm(self.final_df['new_tweets'])]
        
        self.pos_tag_feat(list(self.final_df['new_tweets']))
        
        self.final_df['new_tweets'] = [self.remove_punctuation(i) for i in tqdm(self.final_df['new_tweets'])]
        print("shit")
        self.final_df['new_tweets'] = [self.remove_stopword(i) for i in tqdm(self.final_df['new_tweets'])]
        return self

################################################################################
def printdf(df, rows = 5):
    print(df[:rows].to_markdown())
    return True 
  
#unigrams bow
def bow(X_train, X_test, X_cv, y_train, y_test, y_cv):

    count_vect = CountVectorizer() #in scikit-learn
    count_vect.fit(X_train)

    bow_counts_train = count_vect.transform(X_train)
    bow_counts_cv = count_vect.transform(X_cv)
    bow_counts_test = count_vect.transform(X_test)
    print("the shape of out text BOW vectorizer ",bow_counts_train.get_shape())
    print("the number of unique words ", bow_counts_train.get_shape()[1])
    # final_counts_test=final_counts_test.toarray()
    return count_vect, bow_counts_train, bow_counts_cv, bow_counts_test

#bigrams bow
def bigram(X_train, X_test, X_cv, y_train, y_test, y_cv):
    count_vect = CountVectorizer(ngram_range=(2,2))
    bigram_counts_train = count_vect.fit_transform(X_train)
    bigram_counts_cv = count_vect.transform(X_cv)
    bigram_counts_test = count_vect.transform(X_test)
    print("some feature names ", count_vect.get_feature_names()[:10])
    print("the shape of out  BIGRAM vectorizer ",bigram_counts_train.get_shape())
    print("the number of unique words with bigrams ", bigram_counts_train.get_shape()[1])
    return count_vect, bigram_counts_train, bigram_counts_cv, bigram_counts_test

In [4]:
train = pd.read_csv("data/3a/sentiment_train.csv")
test = pd.read_csv("data/3a/sentiment_test.csv")

# uncomment this later 
# train = train[:30000]
# test = test[:200]

print(train.shape)
print(test.shape)

(1120000, 6)
(480000, 6)
(1120000, 6)
(480000, 6)


In [ ]:
X_train = train["5"]
y_train = train["0"]
X_test = test["5"]
y_test = test['0']

X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.33, random_state=42)


In [15]:
#### FEATURISATION

my_file = Path("data/3a/train_df.csv")
if my_file.is_file():
    print("CSV Found")
    train_df = pd.read_csv("data/3a/train_df.csv")
else:
    print("Generating CSV")
    #featurise train data
    proc = featurise(X_train,y_train).process()
    train_df = proc.final_df
    train_df.to_csv('data/3a/train_df.csv')

my_file = Path("data/3a/val_df.csv")
if my_file.is_file():
    print("CSV Found")
    val_df = pd.read_csv("data/3a/val_df.csv")
else:
    print("Generating CSV")
    #featurise cv data
    proc = featurise(X_cv,y_cv).process()
    val_df = proc.final_df
    val_df.to_csv('data/3a/val_df.csv')

my_file = Path("data/3a/test_df.csv")
if my_file.is_file():
    print("CSV Found")
    test_df = pd.read_csv("data/3a/test_df.csv")
else:
    print("Generating CSV")
    #featurise test data
    proc = featurise(X_test,y_test).process()
    test_df = proc.final_df
    test_df.to_csv('data/3a/test_df.csv')

CSV Found
CSV Found
CSV Found
CSV Found
CSV Found
CSV Found


In [16]:
# train_df.isnull().values.any()
# train_df['new_tweets'].isnull().sum()
# df['your column name'].isnull().sum()
train_df.dropna(subset = ["new_tweets"], inplace=True)
val_df.dropna(subset = ["new_tweets"], inplace=True)
test_df.dropna(subset = ["new_tweets"], inplace=True)

In [17]:
count_vect, bow_counts_train, bow_counts_cv, bow_counts_test = bow(train_df['new_tweets'], test_df['new_tweets'], val_df['new_tweets'], train_df['target'], test_df['target'], val_df['target'])
# count_vect_, bigram_counts_train, bigram_counts_cv, bigram_counts_test = bigram(train_df['new_tweets'], test_df['new_tweets'], val_df['new_tweets'], train_df['target'], test_df['target'], val_df['target'])

the shape of out text BOW vectorizer  (747244, 253751)
the number of unique words  253751
the shape of out text BOW vectorizer  (747244, 253751)
the number of unique words  253751


In [1]:
bow_counts_cv[:10000].toarray()

NameError: name 'bow_counts_cv' is not defined

In [ ]:
# from scipy import sparse
# def save_npz(f,name):
#     sparse.save_npz("data/3a/npz/"+str(name)+".npz", f)
#     return True

# save_npz(bow_counts_cv, "bow_counts_cv")
# save_npz(bow_counts_test, "bow_counts_test")
# save_npz(bow_counts_train, "bow_counts_train")
# save_npz(bigram_counts_train, "bigram_counts_train")
# save_npz(bigram_counts_test, "bigram_counts_test")
# save_npz(bigram_counts_cv, "bigram_counts_cv")

# # train_df.to_csv('data/3a/gram_traincsv.csv')
# # val_df.to_csv('data/3a/gram_valcsv.csv')
# # test_df.to_csv('data/3a/gram_testcsv.csv')